In [2]:
import mechanicalsoup, bs4, re, lxml, json, chardet, unidecode

In [3]:
browser = mechanicalsoup.Browser()

In [4]:
def getSoup(url):
    page = browser.get(url)
    soup = bs4.BeautifulSoup(page.content,"lxml")
    return soup

### Coordinates 
N 43.45655 W 1.01695

In [5]:
def makeSquare(bounds,supp=1):
    coords=[]
    lat1=bounds["northeast"]["lat"]
    long1=bounds["northeast"]["lng"]
    lat2=bounds["southwest"]["lat"]
    long2=bounds["southwest"]["lng"]
    h=lat1-lat2
    l=long1-long2
    hs=(lat1+lat2)/2-h
    lw=(long1+long2)/2-supp*l
    for n in range(2*supp+1):
        for w in range(2*supp+1):
            coords.append((round(hs+n*h,7),round(lw+w*l,7)))
    return coords

In [6]:
def makeHVLine(locCenter,deltas,supp=1,hollow=True):
    gauche=[]
    droite=[]
    haut=[]
    bas=[]
    lat=locCenter["lat"]
    lng=locCenter["lng"]
    h,l=deltas
    for i in range(supp):
        n=i+1
        gauche.append((lat,lng-n*l))
        droite.append((lat,lng+n*l))
        haut.append((lat+n*h,lng))
        bas.append((lat-n*h,lng))
    if hollow:
        return gauche+haut+droite+bas
    else:
        return gauche+haut+[center]+droite+bas
    
def makeCross(bounds,supp=1,hollow=True):
    coords=[]
    locCenter={}
    lat1=bounds["northeast"]["lat"]
    long1=bounds["northeast"]["lng"]
    lat2=bounds["southwest"]["lat"]
    long2=bounds["southwest"]["lng"]
    h=lat1-lat2
    l=long1-long2
    locCenter["lat"]=round((lat1+lat2)/2,7)
    locCenter["lng"]=round((long1+long2)/2,7)
    coords=makeHVLine(locCenter,(h,l),supp,hollow)
    return coords

In [7]:
def convert_fake_unicode_to_real_unicode(string):
    '''https://stackoverflow.com/questions/9973815/python-to-show-special-characters'''
    try:
        return ''.join(map(chr, map(ord, string))).decode('utf-8')
    except:
        return string
    

def getWords(words):
    result=words
    try:
        result=convert_fake_unicode_to_real_unicode(words)
    except UnicodeDecodeError:
        result=words
    return result

In [8]:
def getCoordsMots(origine,rayon,forme="O"):
    mots=[]
    url="https://api.what3words.com/v2/reverse?coords=%f,%f&display=full&lang=fr&format=json&key=QBH16AWV"%origine
    jsonResult=getSoup(url).text
    center=json.loads(jsonResult)
    cWords=center["words"]
    print getWords(cWords)
    print
    if forme=='+':
        wps=makeCross(center["bounds"],rayon)
    elif forme=='x':
        wps=makeCross(center["bounds"],rayon,hollow=False)
    else:
        wps=makeSquare(center["bounds"],rayon)
    for wp in wps:
        url="https://api.what3words.com/v2/reverse?coords=%f,%f&display=full&lang=fr&format=json&key=QBH16AWV"%wp
        jsonResult=getSoup(url).text
        result=json.loads(jsonResult)
        rWords=result["words"]
        words=getWords(rWords)
        mots.append(words)
    return mots

In [9]:
def getMotsMots(w3w,rayon,forme="O",lang="fr"):
    mots=[]
    url="https://api.what3words.com/v2/forward?addr=%s&display=full&format=json&key=QBH16AWV"%w3w
    jsonResult=getSoup(url).text
    center=json.loads(jsonResult)
    cCoords=center["geometry"]
    print cCoords["lat"],cCoords["lng"]
    print
    if forme=='+':
        wps=makeCross(center["bounds"],rayon)
    elif forme=='x':
        wps=makeCross(center["bounds"],rayon,hollow=False)
    else:
        wps=makeSquare(center["bounds"],rayon)
    for wp in wps:
        url="https://api.what3words.com/v2/reverse?coords=%s&display=full&lang=%s&format=json&key=QBH16AWV"%("%f,%f"%wp,lang)
        jsonResult=getSoup(url).text
        result=json.loads(jsonResult)
        rWords=result["words"]
        words=getWords(rWords)
        mots.append(words)
    return mots

In [10]:
rayon=3
origine=(43.456541,-1.016953)
origine=(43.162483, -1.239617)
origine=(43.251750, -0.775150)


# w3w=u"battage.faïence.doublon"
#w3w=u"couchant.sevrer.modique"
#w3w=u"aviatrice.optimiste.côte"
#w3w=u"épineux.aboyeur.comparative"
#w3w=u""
#w3w=u"factice.kiosque.affectif"
#w3w=u"lendemain.ramequin.camembert"
#w3w=u"boisson.massif.encadrer"
#w3w=u"maigre.aucun.intrépide"
#w3w=u"calendrier.différer.connexe"
#w3w=u"aussitôt.honneur.chercher"
#w3w=u"rôtisseur.gratinons.patinant"
#w3w=u"fanion.montrant.amenons"
w3w=u"trouver.livrer.revoir"
w3w=u"combien.gardien.tissure"
w3w=u"valeur.séparer.éclair"
w3w=u"réactivons.connexe.réacteur"
w3w=u"enduire.léguons.écurie"
w3w=u"découverte.plombier.picoter"
w3w=u"trembler.suivante.canif"




In [16]:
# print "\n".join(getCoordsMots(origine,4))
# print
print "\n".join(getMotsMots(w3w,2,forme="O",lang="fr"))
lexique=sorted(u".".join(getMotsMots(w3w,1,forme="+")).split(u"."),key=unidecode.unidecode)
print " ".join(lexique)

43.464087 -1.549305

imprimant.rimant.disque
débat.décorons.assoupir
bourdon.targuer.soupape
cirant.tartiner.facturer
déclic.fixement.vivace
chavirer.froideur.laurier
toiture.piéton.moutarde
trembler.suivante.canif
apprécions.mondiaux.bancal
entente.brisons.mouette
pierre.corail.copiage
filtrer.élisant.visant
musicien.trier.replier
déplacer.ourson.sablant
broute.chausson.voici
avouer.saumon.peignoir
accouru.personne.bordant
dortoir.vocaux.gérons
salière.stuc.combien
berçons.azote.rétablir
ventre.lanière.marge
blanche.diluons.bricoler
marteler.mijoter.jambe
jouons.fonçant.profiter
machin.publions.bassin
43.464087 -1.549305

apprécions bancal bourdon mondiaux moutarde musicien piéton replier soupape targuer toiture trier


In [238]:
import itertools as it
import geopy.distance

for m1,m2,m3 in it.permutations(lexique,3):
    mots="%s.%s.%s"%(m1,m2,m3)
    url="https://api.what3words.com/v2/forward?addr=%s&display=full&format=json&key=QBH16AWV"%mots
    jsonResult=getSoup(url).text
    center=json.loads(jsonResult)
    cCoords=center["geometry"]
    distance=geopy.distance.distance(origine,(cCoords["lat"],cCoords["lng"])).km
    if distance<50:
        print mots,cCoords,distance
#    print mots, geopy.distance.distance(origine,(cCoords["lat"],cCoords["lng"])).km

fermons.fêter.ponde {u'lat': 43.496348, u'lng': -1.469057} 36.845227947
laver.brider.grange {u'lat': 43.496294, u'lng': -1.469057} 36.8445244639
ponde.blaguer.cireux {u'lat': 43.474679, u'lng': -1.464113} 36.2413601099
rougeur.vissant.cireux {u'lat': 43.496321, u'lng': -1.469019} 36.8418237885
vrai.blaguer.carotte {u'lat': 43.496321, u'lng': -1.469094} 36.8478480615


In [ ]:
potentiels=[u"méridien.maquette.animant",
            u"trouver.livrer.revoir",
            u"gravure.poème.îlot",
            u"touriste.naître.rieur",
            u"insolite.pesante.espèce",
            u"trouver.livrer.revoir",
            u"touchons.centrer.décodage",
            u"chercher.lorsque.frôlant",
            u"sentant.emmener.espérer",
            u"retirer.pièce.refaire",
            u"tortue.carotte.crevette",
            
           ]